In [0]:
"""
/FileStore/tables/deloitte-aug-2021/movies.csv
/FileStore/tables/deloitte-aug-2021/ratings.csv

/FileStore/tables/deloitte-aug-2021/links.csv
/FileStore/tables/deloitte-aug-2021/tags.csv
"""

Hi

In [0]:
# Data Frame: Structured Data, where we have columns (column name, column type) and then data
# data frame: meta data (Schema) + data rows [RDD] + APIs [filter, where, join, agg, etc]

# load csv file using data frame
# spark - Spark Session, Entry point for Spark SQL [DataFrame etc]
# for each driver program, there will be ONLY ONE SPARK CONTEXT
# for each driver program, there can be ONE or MORE SPARK SESSION

# we don't have schema, but spark dataframe csv driver will build one schema for us
# default csv will fix all columns with string type
tagsDf = spark.read.format("csv")\
         .option("header", True)\
         .load('/FileStore/tables/deloitte-aug-2021/tags.csv')

# ***DON't USE THIS WAY
tagsDf.printSchema()
# access schema using schema
tagsDf.show(5) # shows 5 records

root
-- userId: string (nullable = true)
-- movieId: string (nullable = true)
-- tag: string (nullable = true)
-- timestamp: string (nullable = true)

+------+-------+---------------+----------+
userId|movieId| tag| timestamp|
+------+-------+---------------+----------+
 2| 60756| funny|1445714994|
 2| 60756|Highly quotable|1445714996|
 2| 60756| will ferrell|1445714992|
 2| 89774| Boxing story|1445715207|
 2| 89774| MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows

In [0]:
# to access schema attached to dataframe
tagsDf.schema

# to access the data rows
# dataframe has rdd, data frame/sql is API, where RDD is core
print(tagsDf.rdd.getNumPartitions())
print(tagsDf.rdd.take(5))


1
[Row(userId='2', movieId='60756', tag='funny', timestamp='1445714994'), Row(userId='2', movieId='60756', tag='Highly quotable', timestamp='1445714996'), Row(userId='2', movieId='60756', tag='will ferrell', timestamp='1445714992'), Row(userId='2', movieId='89774', tag='Boxing story', timestamp='1445715207'), Row(userId='2', movieId='89774', tag='MMA', timestamp='1445715200')]

In [0]:
# load data using inferSchema, request Spark itself to analyse and build a schema with proper data types
# ***DON't USE THIS WAY for LARGE FILES, but OK for small files
linksDf = spark.read.format("csv")\
         .option("header", True)\
         .option("inferSchema", True)\
         .load('/FileStore/tables/deloitte-aug-2021/links.csv')

linksDf.printSchema()
linksDf.show(5)

root
-- movieId: integer (nullable = true)
-- imdbId: integer (nullable = true)
-- tmdbId: integer (nullable = true)

+-------+------+------+
movieId|imdbId|tmdbId|
+-------+------+------+
 1|114709| 862|
 2|113497| 8844|
 3|113228| 15602|
 4|114885| 31357|
 5|113041| 11862|
+-------+------+------+
only showing top 5 rows

In [0]:
# define the schema ourself, better approach
from pyspark.sql.types import StructType, LongType,StringType, IntegerType, DoubleType

movieSchema = StructType()\
              .add("movieId", IntegerType(), True)\
              .add("title", StringType(), True)\
              .add("genres", StringType(), True)


movieDf = spark.read.format("csv")\
         .option("header", True)\
         .schema(movieSchema) \
         .load('/FileStore/tables/deloitte-aug-2021/movies.csv')

movieDf.printSchema()
movieDf.show(4)
 

root
-- movieId: integer (nullable = true)
-- title: string (nullable = true)
-- genres: string (nullable = true)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
+-------+--------------------+--------------------+
only showing top 4 rows

In [0]:
# ratingDf usign schema
# how to avoid using \ for new line continuation with paranthesis

ratingSchema = (StructType()
         .add("userId", IntegerType(), True)
         .add("movieId", IntegerType(), True)
         .add("rating", DoubleType(), True)
         .add("timestamp", LongType(), True))


ratingDf = (spark.read.format("csv")
         .option("header", True)
         .schema(ratingSchema)
         .load('/FileStore/tables/deloitte-aug-2021/ratings.csv'))
 
ratingDf.printSchema()
ratingDf.show(4)

root
-- userId: integer (nullable = true)
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)
-- timestamp: long (nullable = true)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
+------+-------+------+---------+
only showing top 4 rows

In [0]:
print("ratingDf partitions", ratingDf.rdd.getNumPartitions())
ratingDf = ratingDf.repartition(16)
print("ratingDf repartitioned ", ratingDf.rdd.getNumPartitions())


ratingDf partitions 1
ratingDf repartitioned 16

In [0]:
print (ratingDf.count())
print (ratingDf.columns)


100836
['userId', 'movieId', 'rating', 'timestamp']

In [0]:
# select shall project specific columns
# DATA FRAME is IMMUTABLE, WE CANNOT ADD, REMOVE, UPDATE content on original dataframe
# we can create/derive new data frame from existing
# every dataframe even if it is derived has its own schema, dataset
df = ratingDf.select("movieId", "rating")
df.printSchema()
df.show(4)

root
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)

+-------+------+
movieId|rating|
+-------+------+
 1| 4.0|
 3| 4.0|
 6| 4.0|
 47| 5.0|
+-------+------+
only showing top 4 rows

In [0]:
# derive a new column, value from existing dataframe
df = ratingDf.withColumn("rating_adjusted", ratingDf.rating + 0.2)
df.show(2)

# the original dataframe is there as is, READ only
ratingDf.show(2)

+------+-------+------+---------+---------------+
userId|movieId|rating|timestamp|rating_adjusted|
+------+-------+------+---------+---------------+
 1| 1| 4.0|964982703| 4.2|
 1| 3| 4.0|964981247| 4.2|
+------+-------+------+---------+---------------+
only showing top 2 rows

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
+------+-------+------+---------+
only showing top 2 rows

In [0]:
ratingDf.withColumnRenamed("rating", "ratings").show(3)

+------+-------+-------+---------+
userId|movieId|ratings|timestamp|
+------+-------+-------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
+------+-------+-------+---------+
only showing top 3 rows

In [0]:
ratingDf.select('*').show(2)

# alias 
ratingDf.select(ratingDf.rating, (ratingDf.rating + 0.2).alias("adjusted") ).show(4)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
+------+-------+------+---------+
only showing top 2 rows

+------+--------+
rating|adjusted|
+------+--------+
 4.0| 4.2|
 4.0| 4.2|
 4.0| 4.2|
 5.0| 5.2|
+------+--------+
only showing top 4 rows

In [0]:
# filter, where are same, just alias function
ratingDf.filter(ratingDf.rating > 4).show(2)
ratingDf.where(ratingDf.rating > 4).show(2)


+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
+------+-------+------+---------+
only showing top 2 rows

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
+------+-------+------+---------+
only showing top 2 rows

In [0]:
# multiple conditions
ratingDf.filter ( (ratingDf.rating >3) & (ratingDf.rating <= 4) ).show(5)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 110| 4.0|964982176|
 1| 235| 4.0|964980908|
+------+-------+------+---------+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import col 

print("Col", col("rating"))
print("col Data", ratingDf.rating)
// print (col("rating") == ratingDf.rating)

Col Column<b'rating'>
col Data Column<b'rating'>
Column<b'(rating = rating)'>

In [0]:
ratingDf.filter ( col("rating") > 4.0).show()

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
 1| 101| 5.0|964980868|
 1| 151| 5.0|964984041|
 1| 157| 5.0|964984100|
 1| 163| 5.0|964983650|
 1| 216| 5.0|964981208|
 1| 231| 5.0|964981179|
 1| 260| 5.0|964981680|
 1| 333| 5.0|964981179|
 1| 362| 5.0|964982588|
 1| 457| 5.0|964981909|
 1| 527| 5.0|964984002|
 1| 553| 5.0|964984153|
 1| 596| 5.0|964982838|
 1| 608| 5.0|964982931|
 1| 661| 5.0|964982838|
 1| 919| 5.0|964982475|
 1| 923| 5.0|964981529|
 1| 940| 5.0|964982176|
+------+-------+------+---------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import asc, desc

# ascending order
ratingDf.sort("rating").show(5)

# ascending order
ratingDf.sort(asc("rating")).show(5)

ratingDf.sort(desc("rating")).show(5)

+------+-------+------+----------+
userId|movieId|rating| timestamp|
+------+-------+------+----------+
 3| 31| 0.5|1306463578|
 3| 914| 0.5|1306463567|
 3| 527| 0.5|1306464275|
 3| 688| 0.5|1306464228|
 3| 720| 0.5|1306463595|
+------+-------+------+----------+
only showing top 5 rows

+------+-------+------+----------+
userId|movieId|rating| timestamp|
+------+-------+------+----------+
 3| 31| 0.5|1306463578|
 3| 914| 0.5|1306463567|
 3| 527| 0.5|1306464275|
 3| 688| 0.5|1306464228|
 3| 720| 0.5|1306463595|
+------+-------+------+----------+
only showing top 5 rows

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 101| 5.0|964980868|
 1| 216| 5.0|964981208|
 1| 151| 5.0|964984041|
 1| 157| 5.0|964984100|
 1| 47| 5.0|964983815|
+------+-------+------+---------+
only showing top 5 rows

In [0]:
# module import alias name
# importing all with alias name
import pyspark.sql.functions as F
 
print(ratingDf.rdd.getNumPartitions())

# the most popular movies, at least rated by 100 users, avg rating should be > 3.5
mostPopularDf = ratingDf\
                .groupBy("movieId")\
                .agg(F.count("userId"), F.avg('rating').alias("avg_rating"))\
                .withColumnRenamed("count(userId)", "total_ratings")\
                .filter( (F.col("total_ratings") >= 100) & (F.col("avg_rating") >= 3.5) )\
                .sort (desc("avg_rating"))


mostPopularDf.printSchema()
mostPopularDf.show(50)


1
root
-- movieId: integer (nullable = true)
-- total_ratings: long (nullable = false)
-- avg_rating: double (nullable = true)

+-------+-------------+------------------+
movieId|total_ratings| avg_rating|
+-------+-------------+------------------+
 318| 317| 4.429022082018927|
 858| 192| 4.2890625|
 2959| 218| 4.272935779816514|
 1221| 129| 4.25968992248062|
 48516| 107| 4.252336448598131|
 1213| 126| 4.25|
 912| 100| 4.24|
 58559| 149| 4.238255033557047|
 50| 204| 4.237745098039215|
 1197| 142| 4.232394366197183|
 260| 251| 4.231075697211155|
 527| 220| 4.225|
 1208| 107| 4.219626168224299|
 2329| 129| 4.217054263565892|
 1196| 211|4.2156398104265405|
 1198| 200| 4.2075|
 1193| 133| 4.203007518796992|
 1089| 131| 4.202290076335878|
 296| 307| 4.197068403908795|
 2571| 278| 4.192446043165468|
 4973| 120| 4.183333333333334|
 356| 329| 4.164133738601824|
 1136| 136| 4.161764705882353|
 593| 279| 4.161290322580645|
 7361| 131|4.1603053435114505|
 3147| 111| 4.148648648648648|
 2028| 188|4.1462765957446805|
 1210| 196| 4.137755102040816|
 4226| 159| 4.122641509433962|
 7153| 185| 4.118918918918919|
 608| 181| 4.116022099447513|
 4993| 198| 4.106060606060606|
 111| 104| 4.105769230769231|
 541| 124| 4.100806451612903|
 1222| 102| 4.098039215686274|
 1258| 109| 4.08256880733945|
 1704| 141| 4.078014184397163|
 79132| 143| 4.066433566433567|
 60069| 104|4.0576923076923075|
 2858| 204| 4.056372549019608|
 1291| 140| 4.046428571428572|
 778| 102| 4.03921568627451|
 1270| 171| 4.038011695906433|
 110| 237| 4.031645569620253|
 5952| 188|4.0212765957446805|
 293| 133| 4.018796992481203|
 68954| 105| 4.004761904761905|
 4995| 123| 4.0|
 1206| 120| 3.995833333333333|
 457| 190|3.9921052631578946|
+-------+-------------+------------------+
only showing top 50 rows

In [0]:
# joins, inner join

mostPopularMoviesDf = mostPopularDf\
                      .join(movieDf, movieDf.movieId == mostPopularDf.movieId)\
                      .select(mostPopularDf.movieId, 'title', 'total_ratings', 'avg_rating', 'genres')

mostPopularMoviesDf.show(10)

+-------+--------------------+-------------+-----------------+--------------------+
movieId| title|total_ratings| avg_rating| genres|
+-------+--------------------+-------------+-----------------+--------------------+
 318|Shawshank Redempt...| 317|4.429022082018927| Crime|Drama|
 858|Godfather, The (1...| 192| 4.2890625| Crime|Drama|
 2959| Fight Club (1999)| 218|4.272935779816514|Action|Crime|Dram...|
 1221|Godfather: Part I...| 129| 4.25968992248062| Crime|Drama|
 48516|Departed, The (2006)| 107|4.252336448598131|Crime|Drama|Thriller|
 1213| Goodfellas (1990)| 126| 4.25| Crime|Drama|
 912| Casablanca (1942)| 100| 4.24| Drama|Romance|
 58559|Dark Knight, The ...| 149|4.238255033557047|Action|Crime|Dram...|
 50|Usual Suspects, T...| 204|4.237745098039215|Crime|Mystery|Thr...|
 1197|Princess Bride, T...| 142|4.232394366197183|Action|Adventure|...|
+-------+--------------------+-------------+-----------------+--------------------+
only showing top 10 rows

In [0]:
# mostPopularMoviesDf.cache()

mostPopularMoviesDf.write.mode('overwrite')\
                   .csv('/FileStore/tables/deloitte-aug-2021/most-popular-movies-csv')

In [0]:
# reduce to single partition and write to file system
mostPopularMoviesDf.coalesce(1)\
                   .write\
                    .option('header', True)\
                    .mode('overwrite')\
                   .csv('/FileStore/tables/deloitte-aug-2021/most-popular-movies-single-csv')

In [0]:

resultFile = sc.textFile('/FileStore/tables/deloitte-aug-2021/most-popular-movies-single-csv')

resultFile.take(5)

Out[43]: ['movieId,title,total_ratings,avg_rating,genres',
 '318,"Shawshank Redemption, The (1994)",317,4.429022082018927,Crime|Drama',
 '858,"Godfather, The (1972)",192,4.2890625,Crime|Drama',
 '2959,Fight Club (1999),218,4.272935779816514,Action|Crime|Drama|Thriller',
 '1221,"Godfather: Part II, The (1974)",129,4.25968992248062,Crime|Drama']

In [0]:
popularMoviesFromResultDf = spark.read.format("csv")\
         .option("header", True)\
         .option("inferSchema", True)\
         .load('/FileStore/tables/deloitte-aug-2021/most-popular-movies-single-csv')

popularMoviesFromResultDf.printSchema()
popularMoviesFromResultDf.show(10)

root
-- movieId: integer (nullable = true)
-- title: string (nullable = true)
-- total_ratings: integer (nullable = true)
-- avg_rating: double (nullable = true)
-- genres: string (nullable = true)

+-------+--------------------+-------------+-----------------+--------------------+
movieId| title|total_ratings| avg_rating| genres|
+-------+--------------------+-------------+-----------------+--------------------+
 318|Shawshank Redempt...| 317|4.429022082018927| Crime|Drama|
 858|Godfather, The (1...| 192| 4.2890625| Crime|Drama|
 2959| Fight Club (1999)| 218|4.272935779816514|Action|Crime|Dram...|
 1221|Godfather: Part I...| 129| 4.25968992248062| Crime|Drama|
 48516|Departed, The (2006)| 107|4.252336448598131|Crime|Drama|Thriller|
 1213| Goodfellas (1990)| 126| 4.25| Crime|Drama|
 912| Casablanca (1942)| 100| 4.24| Drama|Romance|
 58559|Dark Knight, The ...| 149|4.238255033557047|Action|Crime|Dram...|
 50|Usual Suspects, T...| 204|4.237745098039215|Crime|Mystery|Thr...|
 1197|Princess Bride, T...| 142|4.232394366197183|Action|Adventure|...|
+-------+--------------------+-------------+-----------------+--------------------+
only showing top 10 rows